In [1]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction\experiments\mlflowTracking
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [2]:
from secret import Dagshub_PASSWORD

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/GyanPrakashKushwaha/Customer-Churn-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="GyanPrakashKushwaha"
os.environ["MLFLOW_TRACKING_PASSWORD"]= Dagshub_PASSWORD

## Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass(frozen=True)
class MLFlowTrackingConfig:
    mflow_dir : Path
    test_data: Path
    model_obj : str
    metrics_file: str
    params : dict
    mlflow_uri : str
    target_col : str

## configuration 

In [10]:
from churnPredictor.utils import *
from churnPredictor.constants import *

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)

        create_dirs([self.config.artifacts_root])

    def get_mlflow_tracking_config(self) -> MLFlowTrackingConfig:
        config = self.config.mlflow_tracking
        params = self.params.RandomForest
        schema = self.schema.DEPENDET_FEATURES

        create_dirs([self.config.mlfow_dir])

        return MLFlowTrackingConfig(
            mflow_dir=config.mlfow_dir,
            test_data=config.test_data,
            model_obj=config.model_obj_path,
            metrics_file=config.metrics_file_name,
            params=params,
            target_col=schema,
            mlflow_uri='https://dagshub.com/GyanPrakashKushwaha/Customer-Churn-Prediction.mlflow'
        )

        

## component

In [12]:
from churnPredictor.components.model_trainer import ModelTrainer

In [ ]:
class TrackModelPerformance:
    def __init__(self,config:MLFlowTrackingConfig):
        self.config = config

    def 
        